# `ipfs-log`

## Intro

TODO: Explanation of what CRDT is
https://hal.inria.fr/inria-00555588$

TODO: Concepts and Nomenclature: What are heads and tails
The **head** of a log is an entry that is not referenced by any other entry
The **tail** of a log points to another entry that is not in the input array (or no entries?)

TODO: Some diagrams to show how it works, i.e. heads and tails of logs.

## Table of Contents

1. Prerequisites
  1. IPFS Node
  2. Access Controller
  3. Identity
2. Usage
  1. Creating Logs
  2. Reading Logs
  3. Manipulating Logs

In [1]:
// const Log = require("ipfs-log")
const Log = require("./src/log") // if running from within the repo

## Prerequisites

For a minimum viable `ipfs-log`, you need three things: a running IPFS node, an access controller, and an identity.

### IPFS Node

For our examples, we'll switch between a node.js [`js-ipfs`](https://github.com/ipfs/js-ipfs) instance, and a go node using [`js-ipfs-api`](https://github.com/ipfs/js-ipfs-api).

In [2]:
const IPFS = require("ipfs")
const IpfsApi = require("ipfs-api")

In [3]:
// Default config
var ipfs = new IPFS()
var ipfsapi = new IpfsApi("localhost", 5001)

^ Always a good sign.

### The Access Controller

TODO: Minimum viable details here

In [4]:
const AccessController = require('./src/default-access-controller')

In [5]:
var testACL = new AccessController()

### The Identity

TODO: Minimum viable details here

In [6]:
const IdentityProvider = require('orbit-db-identity-provider')
const Keystore = require('orbit-db-keystore')

In [7]:
const keystore = Keystore.create("./test/fixtures/keys")
const identitySignerFn = async (id, data) => {
    const key = await keystore.getKey(id)
    return keystore.sign(key, data)
}

let testIdentityA, testIdentityB;
(async () => { 
    testIdentityA = await IdentityProvider.createIdentity(keystore, 'userA', identitySignerFn)
    testIdentityB = await IdentityProvider.createIdentity(keystore, 'userB', identitySignerFn)
    testIdentityC = await IdentityProvider.createIdentity(keystore, 'userC', identitySignerFn)
})()

## Usage

With the following, we can create a minimum viable log:
* a working IPFS node
* access controller
* an identity

The full signature is 

```javascript
new Log(ipfs, access, identity, [logId], [entries], [heads], [clock])
```

We'll more into the optional params now. For now, let's create a couple "minimum viable" logs.

### Creating Logs

In [8]:
var log = new Log(ipfs, testACL, testIdentityA)   
var logAPI = new Log(ipfsapi, testACL, testIdentityB)

If you don't supply a `logId`, the current javascript timestamp will be used.

In [9]:
log.id

'1541822346589'

If no log `entries` are specified, the log's `length` will be 0.

In [10]:
logAPI.length

0

`ipfs-log` uses Lamport Clocks, which are a type of vector clock. By tracking both the ID and timestamp of each entry, logs can be merged with other logs and still produce a unique, sorted, set of entries to be processed. This data type is compatible with any "pure" function. 

In [11]:
log.clock

LamportClock {
  id:
   '042750228c5d81653e5142e6a56d5551231649160f77b25797dc427f8a5b2afd650acca10182f0dfc519dc6d6e5216b9a6612dbfc56e906bdbf34ea373c92b30d7',
  time: 0 }

In [12]:
logAPI.clock

LamportClock {
  id:
   '04e9224ee3451772f3ad43068313dc5bdc6d3f2c9a8c3a6ba6f73a472d5f47a96ae6d776de13f2fc2076140fd68ca900df2ca4862b06192adbf8f8cb18a99d69aa',
  time: 0 }

### Creating Entries

Now that we have our logs, let's create some entries!

TODO: Entry Signature

In [13]:
const Entry = require("./src/entry")
const Clock = require('./src/lamport-clock')

### Adding entries at log creation


By utilizng the first of our optional params, we can create a log with entries right from the start. Observe how the length and clock values change. Please note that with this method you have to _manually_ set the clock values for each entry.

In [14]:
let log3;

(async() => {
    var entry1 = await Entry.create(ipfs, testIdentityC, 'C', 'entry1', [], new Clock('C', 0))
    var entry2 = await Entry.create(ipfs, testIdentityC, 'C', 'entry2', [], new Clock('C', 1))
    log3 = new Log(ipfs, testACL, testIdentityC, null, [entry1, entry2])
})()

Swarm listening on /ip4/127.0.0.1/tcp/4003/ws/ipfs/QmTnrsWEWwXhAWP1YEKvC3qQUt3MYprXmTmoRmDY4bwwBj
Swarm listening on /ip4/127.0.0.1/tcp/4002/ipfs/QmTnrsWEWwXhAWP1YEKvC3qQUt3MYprXmTmoRmDY4bwwBj
Swarm listening on /ip4/10.1.10.127/tcp/4002/ipfs/QmTnrsWEWwXhAWP1YEKvC3qQUt3MYprXmTmoRmDY4bwwBj
Swarm listening on /ip4/172.17.0.1/tcp/4002/ipfs/QmTnrsWEWwXhAWP1YEKvC3qQUt3MYprXmTmoRmDY4bwwBj


In [15]:
log3.length

2

In [16]:
log3.clock

LamportClock {
  id:
   '045d10320c2a75982d55e7e487db235341ac0a09a36252de6f3e959b9e249841a4bf4cfb909ec8c801ceeb0679586312e1830a753800ee351da54eb20e401df592',
  time: 1 }

### Adding entries after log creation

Note that the clock in the entries will be ignored and are therefor not necessary for this method.

In [17]:
(async () => {
    var entry3 = await Entry.create(ipfs, testIdentityC, 'C', 'entry3', [])
    var entry4 = await Entry.create(ipfs, testIdentityC, 'C', 'entry4', [])
    var entry5 = await Entry.create(ipfs, testIdentityC, 'C', 'entry5', [])
    log3.append(entry3)
    log3.append(entry4)
    log3.append(entry5)
})()

In [18]:
log3.length

5

In [20]:
log3.clock

LamportClock {
  id:
   '045d10320c2a75982d55e7e487db235341ac0a09a36252de6f3e959b9e249841a4bf4cfb909ec8c801ceeb0679586312e1830a753800ee351da54eb20e401df592',
  time: 4 }

In [24]:
log3._entryIndex

{ QmaDCbLDqxNN1GChsZj1NcS59VmHkJHEFJMmN2gYJqxGpN:
   { hash: 'QmaDCbLDqxNN1GChsZj1NcS59VmHkJHEFJMmN2gYJqxGpN',
     id: 'C',
     payload: 'entry1',
     next: [],
     v: 0,
     clock: LamportClock { id: 'C', time: 0 },
     key:
      '045d10320c2a75982d55e7e487db235341ac0a09a36252de6f3e959b9e249841a4bf4cfb909ec8c801ceeb0679586312e1830a753800ee351da54eb20e401df592',
     identity:
      { id: 'userC',
        publicKey:
         '045d10320c2a75982d55e7e487db235341ac0a09a36252de6f3e959b9e249841a4bf4cfb909ec8c801ceeb0679586312e1830a753800ee351da54eb20e401df592',
        signatures: [Object],
        type: [AsyncFunction: identitySignerFn] },
     sig:
      '30440220600400ab99a5552725c008660f7135731a83e4795f41c42e128fc8410a506a9602204603dab64bc12d2cc9e7330c10e45c6a6dbe7bb62cc830fdd0aac50e743e8ced' },
  QmZ1o6MUxGLHLT7aJQufCmHkaV9eYBynNbkgsADyqn6AG9:
   { hash: 'QmZ1o6MUxGLHLT7aJQufCmHkaV9eYBynNbkgsADyqn6AG9',
     id: 'C',
     payload: 'entry2',
     next: [],
     v: 0,
     clock: 

In [ ]:
// Direct-access Properties
// console.log(log._storage.id().then(console.log)) // The IPFS instance
// console.log(log._access) // AccessController
// console.log(log._identity) // Identity (testIdentity)
// console.log(log._entryIndex) // {} to start
// console.log(log._headsIndex) // {} to start
// console.log(log._nextsIndex) // {} to start

## API Documentation

### Required:
* `entries`
* `heads`


### API Documentation
* Log
  * Constructor
  * get
  * has
  * traverse
  * append
  * join
  * toJSON
  * toSnapshot
  * toBuffer
  * toString
  * isLog
  * toMultihash
  * fromMultihash
  * fromEntryhash
  * fromJSON
  * fromEntry
  * findHeads
  * findTails
  * findTailHashes
  * difference
* LogIO
  * toMultiHash
  * fromMultiHash
  * fromEntryHash
  * fromJSON
  * fromEntry
* LamportClock
  * constructor
  * tick
  * merge
  * clone
  * compare
* Entry
  * create
  * verify
  * toBuffer
  * toMultiHash
  * fromMultiHash
  * isEntry
  * compare
  * isEqual
  * isParent
  * findChildren
* EntryIO
  * fetchParallel
  * fetchAll

* Utility Functions
  * (src/log-sorting.js) LastWriteWins
  * (src/log-sorting.js) SortByClocks
  * (src/log-sorting.js) SortByClockId
  * (src/utils/difference.js) difference
  * (src/utils/find-uniques.js) findUniques
  * (src/utils/intersection.js) intersection
  * (src/utils/is-defined.js) is-defined

* Errors
  * IPFSNotDefinedError
  * LogNotDefinedError
  * NotALogError


## API Documentation

# Log



### Properties

#### id

Returns the ID of the log.

#### values

Returns an `Array` of [entries](https://github.com/haadcode/ipfs-log/blob/master/src/entry.js) in the log.
 The values are in linearized order according to their [Lamport clocks](https://en.wikipedia.org/wiki/Lamp
ort_timestamps).

```javascript
const values = log.values
// TODO: output example
```

#### length

Returns the number of entries in the log.

#### clock

Returns the current timestamp of the log.

#### heads

Returns the heads of the log. Heads are the entries that are not referenced by other entries in the log.

```javascript
const heads = log.heads
// TODO: output example
```

#### tails

Return the tails of the log. Tails are the entries that reference other entries that are not in the log.

```javascript
const tails = log.tails
// TODO: output example
```

### Methods

#### append(data)

Append an entry to the log. Returns a *Promise* that resolves to the updated `Log`.

`ipfs` IPFS instance.

`log` Log to append to.

`data` can be any type of data: Number, String, Object, etc. It can also be an instance of [Entry](https:/
/github.com/haadcode/ipfs-log/blob/master/src/entry.js).

```javascript
log.append({ some: 'data' })
  .then(log => log.append('text'))
  .then(log => console.log(log.values))

// [
//   {
//     hash: 'QmV1KFxZnaguPFp57PMXz5Dd1tf6z9U8csJQoy4xWDzzts',
//     id: 'A',
//     payload: { some: 'data' },
//     next: [],
//     v: 0,
//     clock: LamportClock { id: 'A', time: 0 }
//   },
//   { hash: 'QmSxe4Shd7jt4ExyoBjtvgi1UabNKrZfRJKptwUmSa843u',
//     id: 'A',
//     payload: 'text',
//     next: [ 'QmV1KFxZnaguPFp57PMXz5Dd1tf6z9U8csJQoy4xWDzzts' ],
//     v: 0,
//     clock: LamportClock { id: 'A', time: 1 }
//   }
// ]
```

#### join(log, [length], [id])

Join the log with another log. Returns a Promise that resolves to a `Log` instance. The size of the joined
 log can be specified by giving `length` argument.

```javascript
// log1.values ==> ['A', 'B', 'C']
// log2.values ==> ['C', 'D', 'E']

log1.join(log2)
  .then(() => console.log(log1.values))
// ['A', 'B', 'C', 'D', 'E']
```

### toMultihash()

Writes the log to IPFS and returns the Multihash of the log. Returns a `Promise` that resolves to a Base58
 encoded `string`.

```javascript
log1.toMultihash()
  .then(hash => console.log(hash))

// QmSUrxz12UDsuuQMjzBQ4NDGyYprhFJbQefgeRiomQ5j6T
```

### toBuffer()

Converts the log to a `Buffer` that contains the log as JSON.stringified `string`. Returns a `Buffer`.

```javascript
const buffer = log1.toBuffer()
```

### toString

Returns the log values as a nicely formatted string.

```javascript
console.log(log.toString())
// two
// └─one
//   └─three
```

## Static methods

#### Log.isLog(log)

Check if an object is a `Log` instance.

```javascript
Log.isLog(log1)
// true
Log.isLog('hello')
// false
```

#### Log.expand(ipfs, log, [amount=-1])

Expands a `log` by `amount` by retreiving more entries from the tails of the log. Returns a new `Log` inst
ance.

Expanding a log will retrieve new entries from IPFS, thus causing side effects.

#### Log.expandFrom(ipfs, log, entries, [amount=-1])

Expand a `log` by `amount` by retrieving more entries starting from `entries`. `entries` is an `Array` of
`Entry` instances. Returns a new `Log` instance.

Expanding a log will retrieve new entries from IPFS, thus causing side effects.

#### Log.fromEntry(ipfs, entry, [length=-1])

Create a `Log` from an `Entry`.

Creating a log from an entry will retrieve entries from IPFS, thus causing side effects.

#### Log.toMultihash(ipfs, log)

Returns the multihash of the log.

Converting the log to a multihash will persist the log to IPFS, thus causing side effects.

#### Log.fromMultihash(ipfs, multihash, [length=-1])

Create a `Log` from a multihash.

Creating a log from a multihash will retrieve entries from IPFS, thus causing side effects.